### Music gereration firth Keras and TensorFlow backend

Plan was simple:
1. Read midi file, convert it to matrix of features
2. Create simple model with Keras and LSTM to learn the pattern
3. Use subsample of initial midi file as a input for model to generate pure art
4. Save prediction from model to midi file
.
.
.
5. PROFIT

### Challenges:


We need to solve the following challenges:

1. Reuse the generated information when creating a new file.
2. Training and prediction features must include Velocity, Time, Message Type; and not only note value.

### Notes

1. Time is captured in delta time, i.e. difference in time between the current note, and the one before it. Default value for 1 beat is 96 clocks, time is measured in clocks when viewed with ```mid.print_tracks()```, and as a fraction of a beat when viewed with msg.time (or msg.dict()).

2. We can use ```mid.print_tracks()``` to get a quick overview of the MIDI file

In [1]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

Using TensorFlow backend.


##### Read midi file

In [55]:
mid = MidiFile('Test MIDI Data/random_notes_01.mid') 
notes = []

##### Extract notes sequence

In [56]:
notes_sorted = []
for msg in mid:
    if msg.type == 'note_on' or msg.type == 'note_off':
        notes_sorted.append(msg)
notes_sorted.sort(key=lambda message: message.time)
sorted_list = []
for msg in notes_sorted:
    print(msg.dict())

{'type': 'note_on', 'time': 0, 'channel': 0, 'note': 60, 'velocity': 98}
{'type': 'note_on', 'time': 0, 'channel': 0, 'note': 62, 'velocity': 97}
{'type': 'note_off', 'time': 0, 'channel': 0, 'note': 62, 'velocity': 64}
{'type': 'note_on', 'time': 0, 'channel': 0, 'note': 62, 'velocity': 97}
{'type': 'note_off', 'time': 0, 'channel': 0, 'note': 62, 'velocity': 64}
{'type': 'note_on', 'time': 0, 'channel': 0, 'note': 62, 'velocity': 97}
{'type': 'note_off', 'time': 0, 'channel': 0, 'note': 62, 'velocity': 64}
{'type': 'note_off', 'time': 0.041666666666666664, 'channel': 0, 'note': 62, 'velocity': 64}
{'type': 'note_on', 'time': 0.16666666666666666, 'channel': 0, 'note': 60, 'velocity': 98}
{'type': 'note_on', 'time': 0.1875, 'channel': 0, 'note': 60, 'velocity': 97}
{'type': 'note_on', 'time': 0.20833333333333331, 'channel': 0, 'note': 60, 'velocity': 97}
{'type': 'note_off', 'time': 0.20833333333333331, 'channel': 0, 'note': 60, 'velocity': 64}
{'type': 'note_on', 'time': 0.20833333333

In [58]:
print(mid.print_tracks())

=== Track 0
<meta message track_name name='\x00' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
<message note_on channel=0 note=60 velocity=98 time=0>
<message note_off channel=0 note=60 velocity=64 time=48>
<message note_on channel=0 note=60 velocity=97 time=36>
<message note_off channel=0 note=60 velocity=64 time=48>
<message note_on channel=0 note=60 velocity=97 time=48>
<message note_off channel=0 note=60 velocity=64 time=48>
<message note_on channel=0 note=62 velocity=97 time=44>
<message note_on channel=0 note=60 velocity=97 time=40>
<message note_off channel=0 note=62 velocity=64 time=8>
<message note_off channel=0 note=60 velocity=64 time=40>
<message note_on channel=0 note=60 velocity=98 time=32>
<message note_off channel=0 note=60 velocity=64 time=48>
<message note_on channel=0 note=60 veloci

In [51]:
notes = []
for msg in mid:
   # if not msg.is_meta:
    data = msg.bytes()
    notes.append([data, msg.time, msg.type])
    print(msg.dict())

notes

{'type': 'track_name', 'name': 'MIDI Test\x00', 'time': 0}
{'type': 'time_signature', 'numerator': 4, 'denominator': 4, 'clocks_per_click': 36, 'notated_32nd_notes_per_beat': 8, 'time': 0}
{'type': 'time_signature', 'numerator': 4, 'denominator': 4, 'clocks_per_click': 36, 'notated_32nd_notes_per_beat': 8, 'time': 0}
{'type': 'note_on', 'time': 0, 'channel': 0, 'note': 64, 'velocity': 100}
{'type': 'note_off', 'time': 0.25, 'channel': 0, 'note': 64, 'velocity': 64}
{'type': 'note_on', 'time': 0.25, 'channel': 0, 'note': 64, 'velocity': 100}
{'type': 'note_off', 'time': 0.34375, 'channel': 0, 'note': 64, 'velocity': 64}
{'type': 'note_on', 'time': 0.15625, 'channel': 0, 'note': 64, 'velocity': 100}
{'type': 'note_off', 'time': 0.375, 'channel': 0, 'note': 64, 'velocity': 64}
{'type': 'note_on', 'time': 0.125, 'channel': 0, 'note': 67, 'velocity': 100}
{'type': 'note_on', 'time': 0.125, 'channel': 0, 'note': 65, 'velocity': 100}
{'type': 'note_off', 'time': 0.09375, 'channel': 0, 'note':

[[[255, 3, 10, 77, 73, 68, 73, 32, 84, 101, 115, 116, 0], 0, 'track_name'],
 [[255, 88, 4, 4, 2, 36, 8], 0, 'time_signature'],
 [[255, 88, 4, 4, 2, 36, 8], 0, 'time_signature'],
 [[144, 64, 100], 0, 'note_on'],
 [[128, 64, 64], 0.25, 'note_off'],
 [[144, 64, 100], 0.25, 'note_on'],
 [[128, 64, 64], 0.34375, 'note_off'],
 [[144, 64, 100], 0.15625, 'note_on'],
 [[128, 64, 64], 0.375, 'note_off'],
 [[144, 67, 100], 0.125, 'note_on'],
 [[144, 65, 100], 0.125, 'note_on'],
 [[128, 67, 64], 0.09375, 'note_off'],
 [[128, 65, 64], 0.15625, 'note_off'],
 [[144, 64, 100], 0.125, 'note_on'],
 [[128, 64, 64], 0.25, 'note_off'],
 [[144, 64, 100], 0.25, 'note_on'],
 [[128, 64, 64], 0.34375, 'note_off'],
 [[144, 64, 100], 0.15625, 'note_on'],
 [[128, 64, 64], 0.375, 'note_off'],
 [[144, 67, 100], 0.125, 'note_on'],
 [[144, 65, 100], 0.125, 'note_on'],
 [[128, 67, 64], 0.09375, 'note_off'],
 [[144, 64, 100], 0.15625, 'note_on'],
 [[128, 65, 64], 0, 'note_off'],
 [[128, 64, 64], 0.25, 'note_off'],
 [[14

#### Apply min-max scalling

In [4]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))
notes

[array([0.33333333]),
 array([0.66666667]),
 array([0.30555556]),
 array([0.63888889]),
 array([0.25]),
 array([0.58333333]),
 array([0.19444444]),
 array([0.52777778]),
 array([0.33333333]),
 array([0.66666667]),
 array([0.19444444]),
 array([0.52777778]),
 array([0.25]),
 array([0.58333333]),
 array([0.19444444]),
 array([0.52777778]),
 array([0.30555556]),
 array([0.63888889]),
 array([0.19444444]),
 array([0.52777778]),
 array([0.25]),
 array([0.58333333]),
 array([0.27777778]),
 array([0.61111111]),
 array([0.30555556]),
 array([0.63888889]),
 array([0.19444444]),
 array([0.52777778]),
 array([0.33333333]),
 array([0.66666667]),
 array([0.19444444]),
 array([0.52777778]),
 array([0.33333333]),
 array([0.66666667]),
 array([0.30555556]),
 array([0.63888889]),
 array([0.25]),
 array([0.58333333]),
 array([0.19444444]),
 array([0.52777778]),
 array([0.33333333]),
 array([0.66666667]),
 array([0.19444444]),
 array([0.52777778]),
 array([0.25]),
 array([0.58333333]),
 array([0.19444444

#### Prepare features for training and data subsample for prediction

In [5]:
# LSTM layers requires that data must have a certain shape
# create list of lists fist
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 30

for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = X[:-300]
y = y[:-300]

In [6]:
print(X_test)
print(y)

[[[0.5277777777777777], [0.0], [0.19444444444444442], [0.33333333333333326], [0.38888888888888884], [0.4444444444444444], [0.33333333333333326], [0.5277777777777777], [0.6666666666666667], [0.13888888888888884], [0.33333333333333326], [0.4722222222222221], [0.5277777777777777], [0.5833333333333333], [0.4722222222222221], [0.6666666666666667], [0.8055555555555556], [0.19444444444444442], [0.38888888888888884], [0.5277777777777777], [0.5833333333333333], [0.6388888888888888], [0.5277777777777777], [0.7222222222222221], [0.8611111111111112], [0.05555555555555558], [0.13888888888888884], [0.3055555555555556], [0.38888888888888884], [0.5277777777777777]], [[0.0], [0.19444444444444442], [0.33333333333333326], [0.38888888888888884], [0.4444444444444444], [0.33333333333333326], [0.5277777777777777], [0.6666666666666667], [0.13888888888888884], [0.33333333333333326], [0.4722222222222221], [0.5277777777777777], [0.5833333333333333], [0.4722222222222221], [0.6666666666666667], [0.8055555555555556

#### Made sequential model with several layers, use LSTM as it time dependent data

I also whant to save checkpoints

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
filepath="./Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

#### Train your model.
It might take a while, I was waiting for 1 hour with just 5 epoch

In [9]:
model.fit(np.array(X), np.array(y), 32, 1, verbose=1, callbacks=[model_save_callback])

Epoch 1/1
12770/12770 [==============================] - 51s 4ms/step - loss: 0.0329


#### Make a prediction

In [12]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]
prediction

[58,
 63,
 68,
 62,
 48,
 50,
 53,
 54,
 56,
 57,
 57,
 59,
 59,
 51,
 51,
 53,
 54,
 55,
 57,
 58,
 59,
 50,
 50,
 53,
 55,
 57,
 60,
 66,
 67,
 42,
 48,
 52,
 54,
 56,
 59,
 63,
 67,
 61,
 48,
 50,
 53,
 54,
 56,
 56,
 56,
 57,
 59,
 55,
 51,
 52,
 53,
 56,
 57,
 57,
 54,
 52,
 51,
 53,
 55,
 58,
 57,
 57,
 56,
 52,
 51,
 52,
 53,
 55,
 54,
 56,
 56,
 58,
 55,
 58,
 51,
 54,
 55,
 57,
 53,
 60,
 52,
 63,
 50,
 55,
 55,
 57,
 50,
 59,
 50,
 57,
 51,
 55,
 52,
 55,
 52,
 56,
 58,
 59,
 51,
 53,
 53,
 55,
 55,
 56,
 57,
 59,
 52,
 55,
 55,
 57,
 44,
 55,
 52,
 56,
 54,
 56,
 60,
 63,
 53,
 49,
 52,
 55,
 56,
 59,
 62,
 64,
 49,
 50,
 53,
 54,
 56,
 58,
 63,
 68,
 62,
 48,
 50,
 53,
 54,
 56,
 57,
 57,
 59,
 59,
 51,
 51,
 53,
 54,
 55,
 57,
 58,
 59,
 50,
 50,
 53,
 55,
 57,
 60,
 66,
 67,
 42,
 48,
 52,
 54,
 56,
 59,
 63,
 67,
 61,
 48,
 50,
 53,
 54,
 56,
 56,
 56,
 57,
 59,
 55,
 51,
 52,
 53,
 56,
 57,
 57,
 54,
 52,
 51,
 53,
 55,
 58,
 57,
 57,
 56,
 52,
 51,
 52,
 53,
 55,
 54,


#### Save your result to new midi file

In [11]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velocity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    # Time information challenge
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('LSTM_music.mid')

#### Just listen to it. The result is surreal!